In [2]:
import pandas as pd
import numpy as np

from pandas import DataFrame, Series

# 時系列データ - データ処理

 - [10.5 期間を使った算術演算](#10.5-期間を使った算術演算)
     - 10.5.1 期間頻度の変換
     - 10.5.2 四半期の頻度
     - 10.5.3 タイムスタンプから期間への変換(とその逆)
     - 10.5.4 配列からPeriodIndexを作成する
 - 10.6 再サンプリングと頻度変換
     - 10.6.1 ダウンサンプリング
     - 10.6.2 アップサンプリングと穴埋め
     - 10.6.3 期間で再サンプリングする

---
# 10.5 期間を使った算術演算
**期間**は、日、月、四半期、年など、一定の期間を表現する。`Period`クラスはこのようなデータ型を表現している。

### `Period`クラス

In [3]:
p = pd.Period(2007, freq='A-DEC')
p

Period('2007', 'A-DEC')

この`Period`オブジェクトは2007年1月1日から2007年12月31日までを含んだ期間を表現している。

In [4]:
p + 5

Period('2012', 'A-DEC')

In [5]:
p - 2

Period('2005', 'A-DEC')

In [6]:
# 2つの期間が同じ頻度を持つ場合は、差を取ると、2つの期間の間に含まれる単位期間の数になる
pd.Period('2014', freq='A-DEC') - p

7

### `pd.period_range()`

In [7]:
# 定期的な範囲の期間は、period_range関数を使って作成することができる。
rng = pd.period_range('1/1/2000', '6/30/2000', freq='M')
rng

PeriodIndex(['2000-01', '2000-02', '2000-03', '2000-04', '2000-05', '2000-06'], dtype='int64', freq='M')

### `PeriodIndex`クラス
`PeriodIndex`クラスは期間のシーケンスを保持している。また、pandasのデータ構造の中で軸やインデックスを提供することができる。

In [8]:
Series(np.random.randn(6), index=rng)

2000-01   -1.652210
2000-02    0.731061
2000-03    1.563472
2000-04   -1.241005
2000-05   -1.242625
2000-06   -0.290899
Freq: M, dtype: float64

In [9]:
values = ['2001Q3', '2002Q2', '2003Q1']
index = pd.PeriodIndex(values, freq='Q-DEC')
index

PeriodIndex(['2001Q3', '2002Q2', '2003Q1'], dtype='int64', freq='Q-DEC')

## 10.5.1 期間頻度の変換: `PeriodもしくはPeriodIndexオブジェクト.asfreq()`
`Period`や`PeriodIndex`オブジェクトは`asfreq`メソッドを使って、別の頻度に変換することができる。例として、１年頻度の期間があることを想定する。

In [10]:
# １年頻度の期間をその１年期間の最初の月か最後の月を指定して、1ヶ月の頻度の期間に変換する
p = pd.Period('2007', freq='A-DEC')

In [11]:
p.asfreq('M', how='start')

Period('2007-01', 'M')

In [12]:
p.asfreq('M', how='end')

Period('2007-12', 'M')

12月以外で年度が終わる**営業年度**の場合は、年度に含まれる1ヶ月の期間が違ったものになる。

In [21]:
p = pd.Period('2007', freq='A-MAR')

In [22]:
p.asfreq('M', 'start')

Period('2006-04', 'M')

In [23]:
p.asfreq('M', 'end')

Period('2007-03', 'M')

高い頻度から低い頻度に変換する場合、短い方の期間がどこに含まれるかで変換後の期間が決まる。例えば、`A-JUN`の頻度では、2007年8月は2008年の期間に含まれる。

In [24]:
p = pd.Period('2007-08', 'M')

In [25]:
p.asfreq('A-JUN')

Period('2008', 'A-JUN')

すべての`PeriodIndex`オブジェクトや時系列オブジェクトは、同じ考え方で変換することができる。

In [26]:
rng = pd.period_range('2006', '2009', freq='A-DEC')

In [27]:
ts = Series(np.random.randn(len(rng)), index=rng)

In [28]:
ts

2006   -0.577602
2007    0.863356
2008    0.154318
2009   -0.017096
Freq: A-DEC, dtype: float64

In [29]:
ts.asfreq('M', how='start')

2006-01   -0.577602
2007-01    0.863356
2008-01    0.154318
2009-01   -0.017096
Freq: M, dtype: float64

In [30]:
ts.asfreq('M', how='end')

2006-12   -0.577602
2007-12    0.863356
2008-12    0.154318
2009-12   -0.017096
Freq: M, dtype: float64

## 10.5.2 四半期の頻度: `PeriodもしくはPeriodIndexオブジェクト.asfreq()`
四半期データは会計や金融などの分野では一般的です。多くの四半期報告は、年度の最後の日もしくは最後の営業日である**会計年度末**と関連して報告される。したがって、`2012Q4`という期間は、会計年度末によって意味が違ってくる。pandasは`Q-JAN`から`Q-DEC`まで取り得る１２個すべての四半期の頻度をサポートする。

In [43]:
p = pd.Period('2012Q1', freq='Q-MAR')
p

Period('2012Q1', 'Q-MAR')

In [44]:
p.asfreq('D', 'start')

Period('2011-04-01', 'D')

In [45]:
p.asfreq('D', 'end')

Period('2011-06-30', 'D')

四半期を使った一定の範囲の期間を生成するときは、ご想像の通り、`period_range`メソッドを使う。算術演算についても同様。

In [46]:
rng = pd.period_range('2011Q1', '2017Q4', freq='Q-MAR')

In [47]:
ts = Series(np.arange(len(rng)), index=rng)
ts

2011Q1     0
2011Q2     1
2011Q3     2
2011Q4     3
2012Q1     4
2012Q2     5
2012Q3     6
2012Q4     7
2013Q1     8
2013Q2     9
2013Q3    10
2013Q4    11
2014Q1    12
2014Q2    13
2014Q3    14
2014Q4    15
2015Q1    16
2015Q2    17
2015Q3    18
2015Q4    19
2016Q1    20
2016Q2    21
2016Q3    22
2016Q4    23
2017Q1    24
2017Q2    25
2017Q3    26
2017Q4    27
Freq: Q-MAR, dtype: int64